In [1]:
import pandas as pd
import numpy as np
import sys, zipfile, os, time
import matplotlib
import matplotlib.pyplot as plt
import glob
import datetime
import urllib3
import cdsapi
import xarray
import shutil
import warnings
import pickle
import h5py
import netCDF4
warnings.filterwarnings('ignore')

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua


In [2]:
def get_adj(station, tarvar):
    import datetime
    if tarvar == 'directionbias':
        cdsvar = 'wind_direction'
    elif tarvar == 'uwindbias':
        cdsvar = 'u_component_of_wind'
    elif tarvar == 'vwindbias':
        cdsvar = 'v_component_of_wind'
    else:
        print('not an available wind variable')
        return 0, 0, 0, 0, 0
    """Compare created adjustment to downloaded adjustment
    """
    date0 = datetime.datetime.strptime('1900-01-01', "%Y-%m-%d")

    file = glob.glob('./Wind_adjustment/*/*winddir*'+station+'*')[0]
    data = eua.CDMDataset(file)#.to_dataframe()
    date0 = datetime.datetime.strptime('1900-01-01', "%Y-%m-%d")

    adjdates = []
    basedatum = data.datum[:]
    for i in basedatum:
        adjdates.append(date0 + datetime.timedelta(days=int(i)))
    dates = adjdates
    
    press = data.press[:]
    nightadj = data[tarvar][0,:]
    dayadj = data[tarvar][1,:]    
    
#     c = cdsapi.Client()
#     r = c.retrieve('insitu-comprehensive-upper-air-observation-network',
#                    {'variable': cdsvar,
#                     'optional':['wind_bias_estimate'],
#                     'statid': station,
#                     'skipaa':'8f874f4f',
#                     'pressure_level':[10,20,30,50,70,100,150,200,250,300,400,500,700,850,925,1000]
#                    }
#                   )
#     r.download(target='download.zip')
#     assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
#     z = zipfile.ZipFile('download.zip')
#     z.extractall(path='./tocompare/wind/cds_' + station + '/' + cdsvar)
#     z.close()
#     cdsfile = glob.glob('./tocompare/wind/cds_' + station + '/' + cdsvar + '/*.nc')
#     cdsdata=eua.CDMDataset(cdsfile[0]).to_dataframe()
    data=eua.vm_request_wrapper( {'variable': cdsvar,
                                     'optional':['wind_bias_estimate'],
                                      'statid': station,
                                      'newdl':'000111000',
                                      'pressure_level':[1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,40000,50000,70000,85000,92500,100000]
                                     }, overwrite=True, vm_url='http://srvx8.img.univie.ac.at:8000')
    cdsdata = data.to_dataframe()
    daycdsdata = cdsdata[cdsdata.time.dt.hour > 6][cdsdata.time.dt.hour <= 18]
    nightcdsdata = cdsdata[cdsdata.time.dt.hour <= 6].append(cdsdata[cdsdata.time.dt.hour > 18])
    
    
    calc_day_out = {}
    calc_night_out = {}
    cds_day_out = {}
    cds_night_out = {}
    for i in [10,20,30,50,70,100,150,200,250,300,400,500,700,850,925,1000]:
        calc_day_out[str(i)]=[]
        calc_night_out[str(i)]=[]
        cds_day_out[str(i)]=[]
        cds_night_out[str(i)]=[]
        
    for j in range(len(press)):
        try:
            d_adjd = dayadj[j]
            calc_day_out[str(int(press[j]))].append(d_adjd)
        except:
            calc_day_out[str(int(press[j]))].append(np.nan) 
        try:
            n_adjd = nightadj[j]
            calc_night_out[str(int(press[j]))].append(n_adjd)
        except:
            calc_night_out[str(int(press[j]))].append(np.nan)
                
    return dates, daycdsdata, nightcdsdata, calc_day_out, calc_night_out

In [3]:
def plt_adj(dates, cdata, adata, plev, station, time, adj, save):
#     dates_list = [datetime.datetime.strptime(date, '%Y%m%d').date() for date in dates[1:]]
    fig, ax = plt.subplots()
    ax.scatter(cdata.time, cdata.wind_bias_estimate, label='CDS', s=50)
    ax.scatter(dates, np.asarray(adata[str(plev)]), label='Calc', s=20)
    ax.set(xlabel='time', ylabel='Adjustment: '+adj,
           title= str(station)+' Wind '+str(plev)+' hPa '+time)
    ax.legend()
    ax.grid()
    
    if save:
        fig.savefig(str(station)+'_Wind_'+str(plev)+'_hPa_'+time+'.png')
    plt.show()
    return

In [4]:
# # DOWNLOAD DATA FROM CDS:
# home = os.getcwd()
# %run getwind.py # downloading just: ['0-20001-0-11035', '0-20001-0-10393', '0-20001-0-70219', '0-20000-0-91413', '0-20000-0-35229', '0-20000-0-68994']
# os.chdir(home)

In [5]:
home = os.getcwd()
# RUN ADJUSTMENTS
os.chdir('/raid/home/srvx7/lehre/users/a1400070/CEUAS/CEUAS/public/adjust/Wind_adjustment/')
import adjust_wind_local as aw
via_backend=False
aw.homogenize_winddir(via_backend, fns=['0-20000-0-91413'])# fns=['0-20001-0-11035', '0-20001-0-10393', '0-20001-0-70219', '0-20000-0-91413', '0-20000-0-35229', '0-20000-0-68994'])
# output -> home/Wind_adjustments/0*/feedbackmergedwinddir0*.nc
os.chdir('/raid/home/srvx7/lehre/users/a1400070/CEUAS/CEUAS/public/adjust')

0  duplicates
/raid/home/srvx7/lehre/users/a1400070/CEUAS/CEUAS/public/adjust/Wind_adjustment
/raid/home/srvx11/lehre/users/a1400070/CEUAS/CEUAS/public/adjust/Wind_adjustment/0-20000-0-91413_CEUAS_merged_v1.nc
File: <HDF5 file "0-20000-0-91413_CEUAS_merged_v1.nc" (mode r)>
Permissions: RW (mblaschek)
Filesize: 445.10 MB
Filename: /raid/home/srvx11/lehre/users/a1400070/CEUAS/CEUAS/public/adjust/Wind_adjustment/0-20000-0-91413_CEUAS_merged_v1.nc
(G)roups/(V)ariables: 

 - G | advanced_homogenisation______________________ : : 6
 - G | crs__________________________________________ : : 4
 - V | dateindex____________________________________ : : (26112,)
 - G | era5fb_______________________________________ : : 72
 - G | header_table_________________________________ : : 56
 - G | observations_table___________________________ : : 50
 - G | observed_variable____________________________ : : 9
 - G | recordindices________________________________ : : 18
 - G | sensor_configuration__________________

Data not sorted !!!


dates not None
da False


Not all input dates are found (54883,)


write: 23.56701970100403
wrote ./091413/feedbackmergedwinddir091413.nc
23.76658844947815 1


In [9]:
file = eua.CDMDataset(glob.glob('./Wind_adjustment/0-20000-0-91413_CEUAS_merged_v1.nc')[0])

In [12]:
file.advanced_homogenisation.wind_bias_estimate[:] == file.advanced_homogenisation.wind_bias_estimate_2[:]

array([False, False, False, ..., False, False, False])

In [15]:
a1 = file.advanced_homogenisation.wind_bias_estimate[:]

In [16]:
a2 = file.advanced_homogenisation.wind_bias_estimate_2[:]

In [26]:
test = pd.DataFrame({'old':a1,'new':a2} )
test

,old,new
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
16217121,NaN,NaN
16217122,NaN,NaN
16217123,NaN,NaN
16217124,NaN,NaN


In [28]:
test2 = test.dropna()

In [31]:
test2

,old,new
8436383,0.034353,0.034353
8436384,0.088426,0.088427
8436385,0.000387,0.000387
8436386,0.000774,0.000774
8436387,0.000903,0.000903
...,...,...
12794733,0.000000,0.000000
12794734,0.000000,0.000000
12794735,0.000000,0.000000
12794736,0.000000,0.000000


In [32]:
booldf = test2.old == test2.new

In [36]:
difftest = test2[booldf == False]

In [38]:
diff = difftest.old - difftest.new

In [39]:
np.max(diff)

3.814697265625e-06

In [40]:
np.min(diff)

-7.62939453125e-06

In [41]:
file = eua.CDMDataset(glob.glob('/raid60/scratch/leo/scratch/converted_v6/*35229_CEUAS_merged_v1.nc')[0])

In [80]:
file.recordindices

recordindices:

0_________________________________________________ : : (60225,)
104_______________________________________________ : : (60225,)
105_______________________________________________ : : (60225,)
106_______________________________________________ : : (60225,)
107_______________________________________________ : : (60225,)
117_______________________________________________ : : (60225,)
136_______________________________________________ : : (60225,)
137_______________________________________________ : : (60225,)
138_______________________________________________ : : (60225,)
139_______________________________________________ : : (60225,)
140_______________________________________________ : : (60225,)
34________________________________________________ : : (60225,)
36________________________________________________ : : (60225,)
38________________________________________________ : : (60225,)
39________________________________________________ : : (60225,)
85______________________

In [81]:
file.recordindices['106'][:]

array([8342788, 8342788, 8342788, ..., 9669869, 9669908, 9669943])

In [82]:
file.observations_table.observed_variable[8342788:9669943]

array([106, 106, 106, ..., 106, 106, 106])

In [83]:
wd_v6 = file.observations_table.observation_value[8342788:9669943]
wd_v6

array([200., 270.,  70., ..., 270., 270., 250.], dtype=float32)

In [55]:
temp_v6 = file.observations_table.observation_value[4393851:5688478]
temp_v6

array([252.15, 265.15, 278.15, ..., 300.75, 301.55, 304.55], dtype=float32)

In [53]:
data=eua.vm_request_wrapper( {'variable': 'temperature', 'statid': '35229',
                                     }, overwrite=True, vm_url='http://srvx8.img.univie.ac.at:8000')

In [84]:
wd_data =eua.vm_request_wrapper( {'variable': 'wind_direction', 'statid': '35229',
                                     }, overwrite=True, vm_url='http://srvx8.img.univie.ac.at:8000')

In [88]:
wd_rq = wd_data.wind_from_direction[:]
wd_rq

array([200., 270.,  70., ..., 270., 270., 250.], dtype=float32)

In [60]:
temp_rq = data.ta[:]
temp_rq

array([252.15, 265.15, 278.15, ..., 300.75, 301.55, 304.55], dtype=float32)

In [74]:
df = pd.DataFrame({'idx':range(4393851,5688478),'rq':temp_rq , 'v6':temp_v6})
df

,idx,rq,v6
0,4393851,252.149994,252.149994
1,4393852,265.149994,265.149994
2,4393853,278.149994,278.149994
3,4393854,252.149994,252.149994
4,4393855,265.149994,265.149994
...,...,...,...
1294622,5688473,297.950012,297.950012
1294623,5688474,298.750000,298.750000
1294624,5688475,300.750000,300.750000
1294625,5688476,301.549988,301.549988


In [90]:
wd_df = pd.DataFrame({'idx':range(8342788,9669943),'rq':wd_rq , 'v6':wd_v6})
wd_df

,idx,rq,v6
0,8342788,200.0,200.0
1,8342789,270.0,270.0
2,8342790,70.0,70.0
3,8342791,70.0,70.0
4,8342792,70.0,70.0
...,...,...,...
1327150,9669938,225.0,225.0
1327151,9669939,250.0,250.0
1327152,9669940,270.0,270.0
1327153,9669941,270.0,270.0


In [98]:
wd_df[wd_df.rq != wd_df.v6]

,idx,rq,v6
37609,8380397,NaN,240.0
37610,8380398,NaN,230.0
37611,8380399,NaN,240.0
37612,8380400,NaN,260.0
37613,8380401,NaN,210.0
...,...,...,...
855181,9197969,NaN,250.0
855182,9197970,NaN,245.0
855183,9197971,NaN,240.0
855184,9197972,NaN,245.0


In [101]:
file.observations_table.data_policy_licence[8380397]

4

In [97]:
wd_df.iloc[37609]

idx    8380397.0
rq           NaN
v6         240.0
Name: 37609, dtype: float64

In [95]:
diff = wd_df.rq - wd_df.v6
diff[diff != 0]

37609    NaN
37610    NaN
37611    NaN
37612    NaN
37613    NaN
          ..
855181   NaN
855182   NaN
855183   NaN
855184   NaN
855185   NaN
Length: 929, dtype: float32

In [75]:
df[df.rq != df.v6]

,idx,rq,v6
72332,4466183,NaN,215.350006
72333,4466184,NaN,227.350006
72334,4466185,NaN,234.350006
72335,4466186,NaN,271.950012
72336,4466187,NaN,276.950012
...,...,...,...
869530,5263381,NaN,211.300003
869531,5263382,NaN,201.100006
869532,5263383,NaN,201.500000
869533,5263384,NaN,203.300003


In [79]:
file.observations_table.data_policy_licence[4466182]

0

In [78]:
file.observations_table.data_policy_licence[4466183]

4

In [24]:
type(float('nan'))

float

In [129]:
wd_data =eua.vm_request_wrapper( {'variable': 'temperature', 'statid': ['0-20001-0-11035'], 'date:': '19950101', 'toolbox': 'True'
                                     }, overwrite=True, vm_url='http://srvx8.img.univie.ac.at:8000')

In [130]:
wd_data

File: <HDF5 file "dest_0-20001-0-11035_air_temperature.nc" (mode r)>
Permissions: RW (mblaschek)
Filesize: 216.61 MB
Filename: ./dest_0-20001-0-11035_air_temperature.nc
(G)roups/(V)ariables: 

 - V | lat__________________________________________ : : (6348034,)
 - V | lon__________________________________________ : : (6348034,)
 - V | plev_________________________________________ : : (6348034,)
 - V | report_id____________________________________ : : (6348034, 11)
 - V | station_id___________________________________ : : (6348034, 25)
 - V | ta___________________________________________ : : (6348034,)
 - V | time_________________________________________ : : (6348034,)
 - V | trajectory_index_____________________________ : : (6348034,)
 - V | trajectory_label_____________________________ : : (52346, 11)

In [146]:
c = cdsapi.Client()
r = c.retrieve('insitu-comprehensive-upper-air-observation-network',
               {'variable': 'temperature',
                'statid': ['0-20001-0-11035'],
                'year': '1995',
                'month': '01',
                'day': '01',
                'toolbox': 'True'
               }
              )
r.download(target='download.zip')
assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
z = zipfile.ZipFile('download.zip')
z.extractall(path='./')
z.close()
cdsfile = glob.glob('./*11035*.nc')
cdsdata=eua.CDMDataset(cdsfile[0])

2021-06-16 15:39:46,861 INFO Welcome to the CDS
2021-06-16 15:39:46,861 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-06-16 15:39:47,023 INFO Request is completed
2021-06-16 15:39:47,059 INFO Downloading http://136.156.132.176/cache-compute-0002/cache/data1/adaptor.comprehensive_upper_air.retrieve-1623850293.0264819-14708-5-ee92bc83-4f60-42f3-9ede-d4169b388cfe.zip to download.zip (36.7K)
2021-06-16 15:39:47,144 INFO Download rate 433.5K/s


In [147]:
cdsdata

File: <HDF5 file "dest_0-20001-0-11035_air_temperature.nc" (mode r)>
Permissions: RW (mblaschek)
Filesize: 216.61 MB
Filename: ./dest_0-20001-0-11035_air_temperature.nc
(G)roups/(V)ariables: 

 - V | lat__________________________________________ : : (6348034,)
 - V | lon__________________________________________ : : (6348034,)
 - V | plev_________________________________________ : : (6348034,)
 - V | report_id____________________________________ : : (6348034, 11)
 - V | station_id___________________________________ : : (6348034, 25)
 - V | ta___________________________________________ : : (6348034,)
 - V | time_________________________________________ : : (6348034,)
 - V | trajectory_index_____________________________ : : (6348034,)
 - V | trajectory_label_____________________________ : : (52346, 11)

In [125]:
wd_data =eua.vm_request_wrapper( {'variable': 'wind_direction', 'statid': '68994',
                                     }, overwrite=True, vm_url='http://srvx8.img.univie.ac.at:8000')

In [127]:
wd_data.station_id[0]

array([b'M', b'A', b'R', b'I', b'O', b'N', b' ', b'I', b'S', b'L', b'A',
       b'N', b'D'], dtype='|S1')

In [112]:
file = eua.CDMDataset(glob.glob('/raid60/scratch/leo/scratch/converted_v6/*68994_CEUAS_merged_v1.nc')[0])
print(file.recordindices['106'][:])

wd_v6 = file.observations_table.observation_value[8033086:9356627]
wd_v6

[8033086 8033086 8033086 ... 9356545 9356591 9356627]


array([290., 210., 250., ..., 270., 275., 270.], dtype=float32)

In [113]:
wd_rq = wd_data.wind_from_direction[:]
wd_rq

array([290., 210., 250., ..., 270., 275., 270.], dtype=float32)

In [115]:
wd_df = pd.DataFrame({'idx':range(8033086,9356627),'rq':wd_rq , 'v6':wd_v6})

In [122]:
uneq = wd_df[wd_df.rq != wd_df.v6]

In [124]:
uneq

,idx,rq,v6
22632,8055718,NaN,240.0
22633,8055719,NaN,240.0
22634,8055720,NaN,240.0
28512,8061598,NaN,270.0
28513,8061599,NaN,270.0
...,...,...,...
1262860,9295946,NaN,300.0
1262861,9295947,NaN,300.0
1262862,9295948,NaN,300.0
1266282,9299368,NaN,255.0


In [123]:
uneq.rq.drop_duplicates()

22632   NaN
Name: rq, dtype: float32

In [ ]:
os.getcwd()
os.chdir('/raid/home/srvx7/lehre/users/a1400070/CEUAS/CEUAS/public/adjust')

In [ ]:
station = '68994'
adj = 'uwindbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

In [ ]:
station = '68994'
adj = 'vwindbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

In [ ]:
station = '68994'
adj = 'directionbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

---

In [ ]:
station = '35229'
adj = 'uwindbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

In [ ]:
station = '35229'
adj = 'vwindbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

In [ ]:
station = '35229'
adj = 'directionbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

---

In [ ]:
station = '70219'
adj = 'uwindbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

In [ ]:
station = '70219'
adj = 'vwindbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

In [ ]:
station = '70219'
adj = 'directionbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

---

In [ ]:
station = '10393'
adj = 'uwindbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

In [ ]:
station = '10393'
adj = 'vwindbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

In [ ]:
station = '10393'
adj = 'directionbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

---

In [ ]:
station = '11035'
adj = 'uwindbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

In [ ]:
station = '11035'
adj = 'vwindbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

In [ ]:
station = '11035'
adj = 'directionbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

---

In [ ]:
station = '91413'
adj = 'uwindbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

In [ ]:
station = '91413'
adj = 'vwindbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

In [ ]:
station = '91413'
adj = 'directionbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)